In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_95CI(signal, alpha=5.0):
    """Computes the (default to 95%) confidence intervals of a sequential signal.

    Parameters
    ----------
    signal : {array-like} of shape (n_samples, n_timesteps).
    The matrix of samples. Each row represents a sample, while each column is associated to a timestep.

    alpha : float.
    1 - CI_range, where CI_range represents the range of the confidence interval.

    Returns
    -------
    CI95_lower : {array} of shape (n_timesteps, ).
    The lower bound of the (1 - alpha) confidence interval.

    CI95_upper : {array} of shape (n_timesteps, ).
    The upper bound of the (1 - alpha) confidence interval.
    """
    CI95_lower = []
    CI95_upper = []

    
        
    drawn_at_time_t = signal.copy() # Gather the samples at time t
    lower_p = alpha / 2.0 # Computes the lower bound
    lower = np.percentile(drawn_at_time_t, lower_p) # Retrieves the observation at the lower percentile index
    upper_p = (100 - alpha) + (alpha / 2.0) # Computes the upper bound
    upper = np.percentile(drawn_at_time_t, upper_p) # Retrieves the observation at the upper percentile index

    CI95_lower.append(lower)
    CI95_upper.append(upper)
        
    return lower, upper

In [3]:
filepath = '../data/Données HPV V2.xlsx'

data_HPV_HF = pd.read_excel(filepath, sheet_name='Hautes fréquences')
data_HPV_HF = data_HPV_HF.iloc[:, [1, -3, -8, 5]]
data_HPV_HF = data_HPV_HF.loc[1:]
data_HPV_HF.columns = ['dateStart', 'obs_HPV', 'obs_crAssphages', 'inhibition']

# Manual inputs for dateStart
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('03/07/2024', '2024-07-03')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('01/09/2024', '2024-09-01')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('04/09/2024', '2024-09-04')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('10/08/2024', '2024-08-10')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('14/08/2024', '2024-08-14')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('25/08/2024', '2024-08-25')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('03/11/2024', '2024-11-03')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('06/11/2024', '2024-11-06')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('10/11/2024', '2024-11-10')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('01/12/2024', '2024-12-01')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('04/12/2024', '2024-12-04')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('08/12/2024', '2024-12-08')
data_HPV_HF.dateStart = data_HPV_HF.dateStart.replace('05/01/2025', '2025-01-05')




data_HPV_HF.dateStart = pd.to_datetime(data_HPV_HF.dateStart)#, format='DD/MM/YYYY')
data_HPV_HF.obs_HPV = pd.to_numeric(data_HPV_HF.obs_HPV)
data_HPV_HF.obs_crAssphages = pd.to_numeric(data_HPV_HF.obs_crAssphages)
data_HPV_HF.inhibition = pd.to_numeric(data_HPV_HF.inhibition)
data_HPV_HF.sort_values(by='dateStart', inplace=True)
data_HPV_HF.dateStart.value_counts().head()
data_HPV_HF.drop_duplicates(subset='dateStart', inplace=True)
data_HPV_HF.set_index('dateStart', inplace=True)
data_HPV_HF.head()

,obs_HPV,obs_crAssphages,inhibition
dateStart,,,
2024-01-24,18613.855375,2.663931e+07,0.778384
2024-01-28,32242.166419,5.253436e+07,0.515037
2024-01-31,33190.381878,3.806105e+07,0.479286
2024-02-04,64956.400453,2.717509e+07,0.000000
2024-02-07,51089.113535,2.478178e+07,0.000000


In [4]:
data = pd.read_excel('../data/bootstrap.xlsx', sheet_name='Feuil1')
data.drop('Date', axis=1, inplace=True)
data['dateStart'] = data_HPV_HF.index.tolist()
HPV_types = data.columns[:-1]

In [5]:
frequency_dict = {}
for this_type in HPV_types:
    frequency_dict[this_type] = []

In [6]:
mean_frequency_dict = {}
for this_type in HPV_types:
    mean_frequency_dict[this_type] = data[this_type].sum()

In [7]:
bootstrap_iters = 1000
complete_indexes = data.index.tolist()
RANDOM_SEED = 48 # for reproducibility

for j in range(bootstrap_iters):
    np.random.seed(RANDOM_SEED+j)
    sub_indexes = np.random.choice(complete_indexes, size=len(complete_indexes), replace=True)

    for this_type in list(frequency_dict.keys()):
        these_values = data.loc[sub_indexes, this_type].values
        this_freq = np.sum(these_values)
        frequency_dict[this_type].append(this_freq)

for this_type in list(frequency_dict.keys()):
    frequency_dict[this_type] = np.array(frequency_dict[this_type])

In [8]:
CILs, CIUs = [], []
for this_type in list(mean_frequency_dict.keys()):
    CIL, CIU = get_95CI(frequency_dict[this_type])
    CILs.append(CIL)
    CIUs.append(CIU)

In [9]:
synthesis = pd.DataFrame()
synthesis['HPV_type'] = list(mean_frequency_dict.keys())
synthesis['mean_frequency'] = list(mean_frequency_dict.values())
synthesis['frequency_CIL'] = CILs
synthesis['frequency_CIU'] = CIUs
synthesis.head()

synthesis.to_csv('../outputs/files/bootstrap_frequencies.csv', sep=';', index=False)

In [10]:
### Weekly sample

In [11]:
# extending the previous test to all different subtypes:

input_replicate_1 = pd.read_excel("../data/pvalue-HPV.xlsx", sheet_name='Essai 1')
input_replicate_2 = pd.read_excel("../data/pvalue-HPV.xlsx", sheet_name='Essai 2')

In [12]:
def process_replicate(input_replicate, this_type):

    subcols = ['Date', this_type]
    sub_replicate = input_replicate.loc[::, subcols]
    sub_replicate = sub_replicate.loc[~sub_replicate.Date.isna()]
    
    original_date = ['03/07/2024', '10/08/2024', '14/08/2024',
                    '03/11/2024', '06/11/2024', '10/11/2024',
                    '25/08/2024', '01/09/2024', '04/09/2024',
                    '18/09/2024', '16/10/2024', '01/12/2024',
                    '04/12/2024', '08/12/2024', '01/01/2025',
                    '05/01/2025'
                     ]
    
    modified_date = ['2024-07-03 00:00:00', '2024-08-10 00:00:00', '2024-08-14 00:00:00',
                     '2024-11-03 00:00:00', '2024-11-06 00:00:00', '2024-11-10 00:00:00',
                     '2024-08-25 00:00:00', '2024-09-01 00:00:00', '2024-09-04 00:00:00',
                     '2024-09-18 00:00:00', '2024-10-16 00:00:00', '2024-12-01 00:00:00',
                     '2024-12-04 00:00:00', '2024-12-08 00:00:00', '2025-01-01 00:00:00',
                     '2025-01-05 00:00:00']
    
    for date_index, date in enumerate(original_date):
        sub_replicate.Date = sub_replicate.Date.replace(date, modified_date[date_index])
    
    sub_replicate.Date = pd.to_datetime(sub_replicate.Date)

    # dealing with 2024-08-21 duplicates
    duplicate_indexes = sub_replicate.loc[sub_replicate.Date=='2024-08-21'].index.values
    
    if sub_replicate.loc[(sub_replicate.Date=='2024-08-21')&(~sub_replicate[this_type].isna())].shape[0]==2:
        sub_replicate.drop(duplicate_indexes[-1], axis=0, inplace=True)

    elif sub_replicate.loc[(sub_replicate.Date=='2024-08-21')&(~sub_replicate[this_type].isna())].shape[0]==0:
        sub_replicate.drop(duplicate_indexes[-1], axis=0, inplace=True)

    elif sub_replicate.loc[(sub_replicate.Date=='2024-08-21')&(~sub_replicate[this_type].isna())].shape[0]==1:
        this_index = sub_replicate.loc[(sub_replicate.Date=='2024-08-21')&(~sub_replicate[this_type].isna())].index.tolist()[0]
        dropme = duplicate_indexes[np.where(duplicate_indexes!=this_index)[0][0]]
        sub_replicate.drop(dropme, axis=0, inplace=True)

    sub_replicate = sub_replicate.rename(columns={'Date':'dateStart'})
    sub_replicate.set_index('dateStart', inplace=True)
    
    return sub_replicate

In [13]:
def join_and_get_samples(this_type):
    
    sr1_6 = process_replicate(input_replicate_1, this_type)
    sr2_6 = process_replicate(input_replicate_2, this_type)

    sr_6 = sr1_6.join(sr2_6, lsuffix='_1', rsuffix='_2')

    sr_6.reset_index(inplace=True)
    sr_6['year'] = sr_6.dateStart.dt.year
    sr_6['month'] = sr_6.dateStart.dt.month
    sr_6['week'] = sr_6.dateStart.dt.isocalendar().week

    sr_6['year_week'] = sr_6['year'].astype(str) + '-' + sr_6['week'].astype(str)
    sr_6['year_month'] = sr_6['year'].astype(str) + '-' + sr_6['month'].astype(str)

    for column in [this_type + '_1', this_type + '_2']:
        sr_6[column].replace(' NA', np.nan, inplace=True)
        sr_6[column].replace('NA ', np.nan, inplace=True)
        sr_6[column] = pd.to_numeric(sr_6[column])
    
    return sr_6

In [14]:
### To account for the resdiual bias (some weeks have 1 sample, one have 3 samples, the rest have 2 samples)
### we gonna take a cut with a sliding window
### say we start with 100 samples
### the biweekly model is gonna cut the 100 samples into 50 pairs
### we then gonna take one index among each pair
### the bimonthly model is gonna cut the 100 samples into 25 set of 4
### we gonna take one among 4
### and finally the monthly model is gonna cut the 100 samples into 12 sets of 8 
### lets do this

In [15]:
def get_detection_distribution_weekly(df_replicates):
    bootstrap_iters = 1000
    RANDOM_SEED = 48 # for reproducibility
    np.random.seed(RANDOM_SEED)

    set_of_indexes = df_replicates.index.values.reshape(-1, 2)
    detection_distribution = []
    for this_bootstrap_iter in range(bootstrap_iters):
        detected = 0
        considered = 0
        for k in range(set_of_indexes.shape[0]):
            considered+=1
            
            eligible_indexes = set_of_indexes[k]
            this_drawn_index = np.random.choice(eligible_indexes, size=1, replace=True)[0]
            these_drawn_values = df_replicates.iloc[this_drawn_index, [1, 2]].values.astype(float)
            
            if ~np.isnan(these_drawn_values).all():
                detected+=1
                
        detection_distribution.append(100 * detected / considered)
    
    detection_distribution = np.array(detection_distribution)

    return detection_distribution

In [16]:
types_list = input_replicate_1.columns[3:].tolist()

In [17]:
distribution_type_dict = {}

for this_type in types_list:    
    df_replicates = join_and_get_samples(this_type)
    this_detection_distribution = get_detection_distribution_weekly(df_replicates)
    distribution_type_dict[this_type] = this_detection_distribution

In [18]:
mean_distributions = []
CIL_distributions = []
CIU_distributions = []

for this_type in list(distribution_type_dict.keys()):

    mean_distribution = distribution_type_dict[this_type].mean()
    CIL, CIU = get_95CI(distribution_type_dict[this_type])

    mean_distributions.append(mean_distribution)
    CIL_distributions.append(CIL)
    CIU_distributions.append(CIU)

In [19]:
synthesis = pd.DataFrame()
synthesis['HPV_type'] = list(distribution_type_dict.keys())
synthesis['mean_frequency'] = mean_distributions
synthesis['frequency_CIL'] = CIL_distributions
synthesis['frequency_CIU'] = CIU_distributions
print(synthesis.head())
synthesis.to_csv('../outputs/files/bootstrap_frequencies_weekly_2.csv', sep=';', index=False)

  HPV_type  mean_frequency  frequency_CIL  frequency_CIU
0    HPV 6          93.090          88.00           98.0
1   HPV 11          21.028          14.00           28.0
2   HPV 16          20.816          14.00           28.0
3   HPV 18           4.944           1.95           10.0
4   HPV 26           4.954           1.95           10.0


In [20]:
def get_detection_distribution_bimonthly(df_replicates):
    bootstrap_iters = 1000
    RANDOM_SEED = 48 # for reproducibility
    np.random.seed(RANDOM_SEED)

    set_of_indexes = df_replicates.index.values.reshape(-1, 4)
    detection_distribution = []
    for this_bootstrap_iter in range(bootstrap_iters):
        detected = 0
        considered = 0
        for k in range(set_of_indexes.shape[0]):
            considered+=1
            
            eligible_indexes = set_of_indexes[k]
            this_drawn_index = np.random.choice(eligible_indexes, size=1, replace=True)[0]
            these_drawn_values = df_replicates.iloc[this_drawn_index, [1, 2]].values.astype(float)
            
            if ~np.isnan(these_drawn_values).all():
                detected+=1
                
        detection_distribution.append(100 * detected / considered)
    
    detection_distribution = np.array(detection_distribution)

    return detection_distribution

In [21]:
distribution_type_dict = {}

for this_type in types_list:
    df_replicates = join_and_get_samples(this_type)
    this_detection_distribution = get_detection_distribution_bimonthly(df_replicates)
    distribution_type_dict[this_type] = this_detection_distribution

In [22]:
mean_distributions = []
CIL_distributions = []
CIU_distributions = []

for this_type in list(distribution_type_dict.keys()):

    mean_distribution = distribution_type_dict[this_type].mean()
    CIL, CIU = get_95CI(distribution_type_dict[this_type])

    mean_distributions.append(mean_distribution)
    CIL_distributions.append(CIL)
    CIU_distributions.append(CIU)

In [23]:
synthesis = pd.DataFrame()
synthesis['HPV_type'] = list(distribution_type_dict.keys())
synthesis['mean_frequency'] = mean_distributions
synthesis['frequency_CIL'] = CIL_distributions
synthesis['frequency_CIU'] = CIU_distributions
print(synthesis.head())
synthesis.to_csv('../outputs/files/bootstrap_frequencies_bimonthly_2.csv', sep=';', index=False)

  HPV_type  mean_frequency  frequency_CIL  frequency_CIU
0    HPV 6          92.996           84.0          100.0
1   HPV 11          20.660            8.0           36.0
2   HPV 16          20.740            8.0           32.1
3   HPV 18           4.956            0.0           12.0
4   HPV 26           5.236            0.0           12.0


In [24]:
def get_detection_distribution_monthly(df_replicates):
    bootstrap_iters = 1000
    RANDOM_SEED = 48 # for reproducibility
    np.random.seed(RANDOM_SEED)

    indexes = df_replicates.index.values
    set_of_indexes = indexes[:len(indexes)//8 * 8].reshape(-1, 8).tolist()
    if len(indexes) % 8:
        set_of_indexes[-1].append(indexes[-1])
    set_of_indexes = np.array(set_of_indexes, dtype=object)

    detection_distribution = []
    for this_bootstrap_iter in range(bootstrap_iters):
        detected = 0
        considered = 0
        for k in range(set_of_indexes.shape[0]):
            considered+=1
            
            eligible_indexes = set_of_indexes[k]
            this_drawn_index = np.random.choice(eligible_indexes, size=1, replace=True)[0]
            these_drawn_values = df_replicates.iloc[this_drawn_index, [1, 2]].values.astype(float)
            
            if ~np.isnan(these_drawn_values).all():
                detected+=1
                
        detection_distribution.append(100 * detected / considered)
    
    detection_distribution = np.array(detection_distribution)

    return detection_distribution

In [25]:
distribution_type_dict = {}

for this_type in types_list:    
    df_replicates = join_and_get_samples(this_type)
    this_detection_distribution = get_detection_distribution_monthly(df_replicates)
    distribution_type_dict[this_type] = this_detection_distribution

In [26]:
mean_distributions = []
CIL_distributions = []
CIU_distributions = []

for this_type in list(distribution_type_dict.keys()):

    mean_distribution = distribution_type_dict[this_type].mean()
    CIL, CIU = get_95CI(distribution_type_dict[this_type])

    mean_distributions.append(mean_distribution)
    CIL_distributions.append(CIL)
    CIU_distributions.append(CIU)

In [27]:
synthesis = pd.DataFrame()
synthesis['HPV_type'] = list(distribution_type_dict.keys())
synthesis['mean_frequency'] = mean_distributions
synthesis['frequency_CIL'] = CIL_distributions
synthesis['frequency_CIU'] = CIU_distributions
print(synthesis.head())
synthesis.to_csv('../outputs/files/bootstrap_frequencies_monthly_2.csv', sep=';', index=False)

  HPV_type  mean_frequency  frequency_CIL  frequency_CIU
0    HPV 6       92.558333           75.0     100.000000
1   HPV 11       21.125000            0.0      41.666667
2   HPV 16       18.383333            0.0      41.666667
3   HPV 18        4.475000            0.0      16.666667
4   HPV 26        5.425000            0.0      16.666667
